In [4]:
import os
import pathlib
import geopandas as gpd

import pandas as pd
import rioxarray as rxr

# Import packages
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

In [1]:
%store -r curl_gdf

In [2]:
model_name = 'BNU-ESM'
variable_name = 'pr'
var_long_name = 'precipitation'
dir_path = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'

In [5]:
# build a directory for the macav2 data

curl_path = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'habitat_suitability'
)
os.makedirs(curl_path, exist_ok=True)
# curl_gdf.to_file('habitat_suitability')
# curl_gdf = gpd.read_file(curl_path)

# Define the file path for the GeoDataFrame
curl_file = os.path.join(curl_path, 'curl_data.shp')

# Save the GeoDataFrame to the file
curl_gdf.to_file(curl_file)

# Read the GeoDataFrame back from the file
curl_gdf = gpd.read_file(curl_file)

In [6]:
def convert_longitude(longitude):
        """Convert longitude from 0-360 to -180-180"""
        return (longitude - 360) if longitude > 180 else longitude

maca_da_list = []
for site_name, site_gdf in {'curl': curl_gdf}.items():
        for variable in ["pr"]:
                for start_year in [2096]:
                        end_year = start_year + 3
                        maca_url = (
                                "http://thredds.northwestknowledge.net:8080/thredds"
                                f"/dodsC/MACAV2/BNU-ESM/macav2metdata_{variable}_BNU-ESM_"
                                f"r1i1p1_rcp85_{start_year}_{end_year}_CONUS_monthly.nc"
                        # "http://thredds/fileServer/MACAV2/BNU-ESM/macav2metdata"
                        # f"_{variable}_BNU-ESM_r1i1p1_rcp45_{start_year}"
                        # f"_{end_year}_CONUS_monthly.nc"
                        )
                        maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                        bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
                        maca_da = maca_da.assign_coords(
                                lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
                        )
                        maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                        maca_da.rio.clip_box(*bounds)
                        maca_da_list.append(dict(
                                site_name=site_name,
                                variable=variable,
                                start_year=start_year,
                                da=maca_da))
pd.DataFrame(maca_da_list)

,site_name,variable,start_year,da
0,curl,pr,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [7]:
maca_df = pd.DataFrame(maca_da_list)
maca_df

,site_name,variable,start_year,da
0,curl,pr,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [8]:
%store maca_df

Stored 'maca_df' (DataFrame)


In [9]:
maca_da.rio.clip_box(*curl_gdf.to_crs(maca_da.rio.crs).total_bounds)
maca_da.rio.clip_box(*bounds)

<xarray.DataArray 'precipitation' (time: 48, lat: 8, lon: 9)> Size: 14kB
[3456 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 64B 42.02 42.06 42.1 42.15 42.19 42.23 42.27 42.31
  * time     (time) object 384B 2096-01-15 00:00:00 ... 2099-12-15 00:00:00
  * lon      (lon) float64 72B -112.9 -112.8 -112.8 ... -112.6 -112.6 -112.5
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [  8 115 276]